In [18]:
from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,55.599998,57.049999,54.500000,57.000000,0.064604,184093
1,2000-01-04,56.500000,56.500000,52.099998,53.000000,0.060071,192154
2,2000-01-05,52.099998,52.099998,50.349998,50.750000,0.057521,86948
3,2000-01-06,50.750000,50.750000,50.750000,50.750000,0.057521,0
4,2000-01-07,50.900002,53.000000,50.049999,52.400002,0.059391,111870


In [8]:
from notebooks.functions import process_stock_data
import pandas as pd
import os

# Lista de nombres de archivos CSV
csv_files = [
    'Acciona_data.csv', 'BP plc_data.csv', 'Enagas_data.csv',  'Exxon Mobil Corporation_data.csv', 'Gazprom_data.csv', 'Honeywell International Inc._data.csv',
    'Iberdrola_data.csv', 'Kinder Morgan, Inc._data.csv',
    'NextEra Energy, Inc._data.csv', 'Repsol_data.csv', 'Saudi Arabian Oil Company (Aramco)_data.csv',
    'Schlumberger Limited_data.csv', 'Solaredge Technologies Inc._data.csv' ,'Solaria_data.csv', 'Técnicas Reunidas_data.csv' 
]


# Procesar cada archivo CSV en la lista
for file_name in csv_files:
    process_stock_data(file_name)

print("Procesamiento completado.")


Procesamiento completado.


In [1]:
from stoks_view import app
app.run_server(mode='inline', debug=True)

C:\Users\pablo\Desktop\DATA SCIENCE COURSE\Finance_EDA\notebooks\stoks_view.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\pablo\Desktop\DATA SCIENCE COURSE\Finance_EDA\notebooks\stoks_view.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.